## Connect to your database

In [4]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

## Testing Retrieve Available Repositories

In [5]:

repolist = pd.DataFrame()

repo_query1 = salc.sql.text(f"""c
                SELECT
                    repo_id,
                    pull_request_labels.pr_src_color,
                    COUNT ( * ) AS repo_color_count
                FROM
                    pull_requests,
                    pull_request_labels
                WHERE
                    pull_requests.pull_request_id = pull_request_labels.pull_request_id --pull_requests.pr_created_at,
                GROUP BY
                    repo_id,
                    pull_request_labels.pr_src_color
                    order by repo_color_count desc;
                    """)

repo_query2 = salc.sql.text(f"""
                SELECT
                    pull_request_labels.pr_src_color,
                    COUNT ( * ) AS repo_color_count
                FROM
                    pull_requests,
                    pull_request_labels
                WHERE
                    pull_requests.pull_request_id = pull_request_labels.pull_request_id --pull_requests.pr_created_at,
                GROUP BY
                    pull_request_labels.pr_src_color
                order by repo_color_count desc;
                """)

repolist = pd.read_sql(repo_query2, con=engine)

display(repolist)

repolist.dtypes-------------------

,pr_src_color,repo_color_count
0,21ef3d,16
1,84b6eb,11
2,D22BF9,9
3,0366d6,8
4,e6e6e6,6
...,...,...
57,5546db,1
58,56a8ff,1
59,58bc0b,1
60,6f31ea,1


pr_src_color        object
repo_color_count     int64
dtype: object